First, import the Python modules necessary for this notebook to run. These can be installed using, for instance, `pip` or `conda`. 

In [3]:
import numpy as np # Handles maths
import matplotlib.pyplot as plt # Handles graphing
import xarray as xr # Helpful for spatial data
import requests # Downloads webpages
from bs4 import BeautifulSoup

In [40]:
url = "http://epbcnotices.environment.gov.au/publicnoticesreferrals/#"

In [61]:
local = "/home/student.unimelb.edu.au/shorte1/Documents/ACF_consulting/test.html"

In [64]:
# example = urllib.request.urlopen(local).read()
example_soup = BeautifulSoup(open(local))

'utf-8'

In [39]:
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(options=options)
driver.get(url)
time.sleep(3)
page = driver.page_source
driver.quit()
soup = BeautifulSoup(page, 'html.parser')
container = soup.find_all('div')
print(container)

WebDriverException: Message: unknown error: Runtime.executionContextCreated has invalid 'context': {"auxData":{"frameId":"2F25DFB6880F24D774518BCBF77C3EB7","isDefault":true,"type":"default"},"id":1,"name":"","origin":"://"}
  (Session info: chrome=88.0.4324.182)
  (Driver info: chromedriver=2.12.301324 (de8ab311bc9374d0ade71f7c167bad61848c7c48),platform=Linux 4.15.0-135-generic x86_64)


In [65]:
print(example_soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html>
<!-- saved from url=(0062)http://epbcnotices.environment.gov.au/publicnoticesreferrals/# --><html lang="en"><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><title>
	Public notices - referrals · Basic Portal
</title>
<script type="text/javascript">
			// Fix for incorrect viewport width setting in IE 10 on Windows Phone 8.
			if (navigator.userAgent.match(/IEMobile\/10\.0/)) {
				var msViewportStyle = document.createElement("style");
				msViewportStyle.appendChild(document.createTextNode("@-ms-viewport{width:auto!important}"));
				document.getElementsByTagName("head")[0].appendChild(msViewportStyle);
}

		</script>
<script src="./Public notices - referrals · Basic Portal_files/default.preform.bundle.js"></script>
<link href="./Public notices - referrals · Basic Portal_files/bootstrap.min.cs

In [66]:
list(example_soup.children)

['html',
 ' saved from url=(0062)http://epbcnotices.environment.gov.au/publicnoticesreferrals/# ',
 <html lang="en"><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><title>
 	Public notices - referrals · Basic Portal
 </title>
 <script type="text/javascript">
 			// Fix for incorrect viewport width setting in IE 10 on Windows Phone 8.
 			if (navigator.userAgent.match(/IEMobile\/10\.0/)) {
 				var msViewportStyle = document.createElement("style");
 				msViewportStyle.appendChild(document.createTextNode("@-ms-viewport{width:auto!important}"));
 				document.getElementsByTagName("head")[0].appendChild(msViewportStyle);
 }
 
 		</script>
 <script src="./Public notices - referrals · Basic Portal_files/default.preform.bundle.js"></script>
 <link href="./Public notices - referrals · Basic Portal_files/bootstrap.min.css" rel="stylesheet"/>
 <l

In [215]:
[type(item) for item in list(example_soup.children)]

[bs4.element.Doctype,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Comment,
 bs4.element.NavigableString]

In [216]:
list(example_soup.children)

['html', <html lang="en">
 <head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><title>
 	Public notices - referrals · Basic Portal
 </title>
 <script type="text/javascript">
 			// Fix for incorrect viewport width setting in IE 10 on Windows Phone 8.
 			if (navigator.userAgent.match(/IEMobile\/10\.0/)) {
 				var msViewportStyle = document.createElement("style");
 				msViewportStyle.appendChild(document.createTextNode("@-ms-viewport{width:auto!important}"));
 				document.getElementsByTagName("head")[0].appendChild(msViewportStyle);
 }
 
 		</script>
 <script src="/js/default.preform.bundle.js?v=Ztx15Sr8IUQwRBhAs6594fCXKUSFT98JdLezjlw0oP81"></script>
 <link href="/css/bootstrap.min.css" rel="stylesheet"/>
 <link href="/css/default.bundle.css?v=RAQSQRk9It3GmJi-lvxETMvaGy3suk6xcMsZNUVLODo1" rel="stylesheet"/>
 <!--[if lt IE 9]>
 				<script src="//html5shim.googlecode.com/svn/trunk/htm

In [217]:
txt = example_soup.prettify()

In [83]:
list(list(list(example_soup.find_all('table'))[0].children)[1].children)

[<tr data-entity="new_casedecision" data-id="824f02a9-2077-eb11-80c5-00505684c563" data-name="2009/5173"><td data-attribute="new_epbcnumber" data-type="System.String" data-value="2009/5173"><a class="details-link has-tooltip launch-modal" data-entityformid="bd49f92c-14e8-431d-bd40-e6fdc206cddb" data-toggle="tooltip" href="http://epbcnotices.environment.gov.au/publicnoticesreferrals/#" title="View Details">2009/5173</a></td><td data-attribute="new_titleforweb" data-type="System.String" data-value="Gladstone Area Water Board and SunWater Limited/Water Management and Use/Lower Fitzroy-Mackenzie Catchment/Queensland/Lower Fitzroy River Infrastructure Project">Gladstone Area Water Board and SunWater Limited/Water Management and Use/Lower Fitzroy-Mackenzie Catchment/Queensland/Lower Fitzroy River Infrastructure Project</td><td data-attribute="new_webdisplayname" data-type="System.String" data-value="Correction notice">Correction notice</td><td data-attribute="new_dateofnotice" data-type="Sys

In [55]:
list(list(list(example_soup.find_all('div', class_="entity-grid entitylist"))[0].children)[0].children)

[]

In [23]:
list(example_soup.find_all('div', class_="view-grid table-responsive"))[0]

<div class="view-grid table-responsive"></div>

In [220]:
list(list(list(example_soup.find_all('div', class_="col-md-9")[0].children)[0].children)[0].children)[0]

<div class="view-grid table-responsive"></div>